In [ ]:
import numpy as np
import pandas as pd
import scipy

from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
#Filter

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs  # Nyquist Frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y


In [ ]:
# Parameters
fs = 256.0       # Sampling frequency of Muse 2, typically 256 Hz
cutoff = 30.0    # Desired cutoff frequency of the high-cut filter in Hz

#Decide on your cutoff frequency based on your analysis needs and the Nyquist criterion.
#The Nyquist frequency is half the sampling rate of your data, and your cutoff should be below this value.

# Apply the filter to each channel
data = df.values
filtered_data = np.zeros_like(data)  # Initialize an array to hold the filtered data

for i in range(data.shape[1]):  # Assuming data.shape[1] is the number of channels
    filtered_data[:, i] = butter_lowpass_filter(data[:, i], cutoff, fs, order=5)

In [ ]:
features = data[:, :5]  # All rows, first 5 columns for features
labels = data[:, 5]     # All rows, 6th column for labels

# Assume features and labels are prepared arrays from your EEG data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy:.2f}")
